<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-12 09:56:24
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15


-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.40 C
-------------------
Today PnL: 48.93 K (0.35%)
Current PnL: -28.18 L (-18.21%)
CY Booked + Current PnL: -13.89 L (-8.98%)
-------------------
Total profit:  1.29 L
Total loss:  -29.47 L
-------------------
Total Booked + Current PnL: 13.09 L (10.24%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.2%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 92.82 L (66.25%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 5.13%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.14,-16.69,20.14,0.09,16909.0,-16819.0,83958.0,94.88,43.0,M-SC,2.38,253.0,-0.99,0.62,10.30,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,0.33,-12.18,13.90,0.02,21307.0,-21270.0,153289.0,133.82,53.0,M-SC,15.84,234.0,-1.00,1.13,33.33,OX40N,NTT,PAINTS
50,MASFIN,398.61,1.30,-0.77,22.99,22.05,22353.0,-750.0,97230.0,-14.63,59.0,H-SC,6.99,164.0,-0.03,0.72,41.33,XR,ATH,FINANCE
43,ITC,452.00,-0.09,-1.64,12.28,10.44,24173.0,-3291.0,196847.0,-41.15,46.0,X-LC,1.82,5.0,-0.14,1.46,3.83,X40,NTT,FMCG
82,VOLTAS,1530.00,0.60,6.51,12.38,19.69,25283.0,12483.0,204225.0,-3.84,51.0,X-MC,2.58,78.0,0.49,1.51,13.75,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,2.46,-13.83,110.83,81.67,89589.0,-12975.0,80835.0,7266.67,54.0,L-SC,13.69,271.0,-0.14,0.60,56.54,XR,NTT,CERAMICS
22,DIXON,18931.72,1.93,-14.98,43.00,21.58,51235.0,-20988.0,119151.0,-57.13,34.0,X-MC,7.75,56.0,-0.41,0.88,7.19,X40N,ATH,IT
10,BANDHANBNK,400.00,1.77,-28.87,170.53,92.42,337874.0,-80427.0,198132.0,39.37,47.0,H-SC,8.78,170.0,-0.24,1.47,14.50,XY24,NTT,BANKS
40,INDUSINDBK,1800.00,1.50,-37.39,112.25,32.89,54261.0,-28868.0,48339.0,-697.12,55.0,L-MC,5.88,259.0,-0.53,0.36,33.12,XR,NTT,BANKS
83,WHIRLPOOL,2270.00,1.32,-25.00,142.55,81.92,122740.0,-28695.0,86103.0,-56.08,24.0,M-SC,13.20,236.0,-0.23,0.64,2.71,XR,NTT,DURABLES


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,GLAXO,3466.20,-1.41,3.77,34.09,39.15,70498.0,7516.0,206800.0,-16.74,54.0,X-MC,11.05,60.0,0.11,1.53,33.87,X40,ATH,PHARMA
56,RAJESHEXPO,518.00,-1.24,-63.29,172.56,0.07,86906.0,-86814.0,50363.0,1745.96,54.0,L-SC,2.39,268.0,-1.00,0.37,25.26,OX40N,NTT,JEWELLERY
64,SHALBY,327.00,-1.12,-25.89,68.80,25.10,92363.0,-46895.0,134248.0,887.24,28.0,M-SC,10.81,250.0,-0.51,0.99,7.59,XY24,NTT,HEALTHCARE
84,WIPRO,420.00,-0.49,5.82,62.80,72.27,112602.0,9855.0,179303.0,-9.26,63.0,M-LC,1.99,99.0,0.09,1.33,12.35,XR,NTT,IT
61,ROUTE,2227.21,-0.47,-48.54,228.50,69.05,156472.0,-64588.0,68478.0,-58.36,47.0,H-SC,23.13,140.0,-0.41,0.51,5.61,SR,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,398.61,1.3,-0.77,22.99,22.05,22353.0,-750.0,97230.0,-14.63,59.0,H-SC,6.99,164.0,-0.03,0.72,41.33,XR,ATH,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.06,-6.40,122.13,107.91,168775.0,-9451.0,138193.0,-23.79,40.0,M-SC,12.49,216.0,-0.06,1.02,3.04,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.33,-12.18,13.90,0.02,21307.0,-21270.0,153289.0,133.82,53.0,M-SC,15.84,234.0,-1.00,1.13,33.33,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.14,-16.69,20.14,0.09,16909.0,-16819.0,83958.0,94.88,43.0,M-SC,2.38,253.0,-0.99,0.62,10.30,OX40N,NTT,DURABLES
47,KANSAINER,340.00,0.66,-24.99,51.29,13.47,103742.0,-67401.0,202266.0,-69.09,34.0,H-SC,4.67,158.0,-0.65,1.50,2.85,XY24,NTT,PAINTS
66,SIS,528.00,0.18,-25.36,62.46,21.26,51764.0,-28157.0,82875.0,1953.37,44.0,H-SC,4.78,166.0,-0.54,0.61,11.97,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-0.49,5.82,62.8,72.27,112602.0,9855.0,179303.0,-9.26,63.0,M-LC,1.99,99.0,0.09,1.33,12.35,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.49,5.82,62.80,72.27,112602.0,9855.0,179303.0,-9.26,63.0,M-LC,1.99,99.0,0.09,1.33,12.35,XR,NTT,IT
50,MASFIN,398.61,1.30,-0.77,22.99,22.05,22353.0,-750.0,97230.0,-14.63,59.0,H-SC,6.99,164.0,-0.03,0.72,41.33,XR,ATH,FINANCE
38,INDIAMART,4810.62,-0.28,-4.07,115.48,106.72,136638.0,-5014.0,118322.0,-52.54,31.0,H-SC,2.31,139.0,-0.04,0.88,18.36,AR,ATH,MISC
37,IEX,219.00,0.84,-2.51,52.45,48.63,102242.0,-5021.0,194933.0,-35.06,53.0,H-SC,14.02,136.0,-0.05,1.44,8.75,XR,NTT,MISC
85,ZYDUSLIFE,1286.17,0.59,-2.14,38.60,35.63,79164.0,-4486.0,205088.0,-15.85,42.0,H-MC,3.11,119.0,-0.06,1.52,14.17,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,0.19,-21.48,89.06,48.45,140216.0,-43066.0,157440.0,-27.18,26.0,X-MC,12.69,64.0,-0.31,1.17,2.51,X40N,ATH,FINANCE
33,HINDUNILVR,2922.00,-0.41,-10.68,27.25,13.67,62571.0,-27443.0,229620.0,-17.28,29.0,X-LC,2.40,14.0,-0.44,1.70,8.40,XY25,NTT,FMCG
75,TMPV,600.00,-0.03,-17.88,73.14,42.18,166020.0,-49420.0,226990.0,-26.74,29.0,X-LC,6.38,3.0,-0.30,1.68,0.92,XY24,NTT,AUTO
53,PAGEIND,51605.08,0.01,-9.44,39.21,26.07,58141.0,-15460.0,148280.0,-33.27,33.0,X-MC,12.39,55.0,-0.27,1.10,1.02,X40,ATH,APPARELS
22,DIXON,18931.72,1.93,-14.98,43.00,21.58,51235.0,-20988.0,119151.0,-57.13,34.0,X-MC,7.75,56.0,-0.41,0.88,7.19,X40N,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.0,0.98,-79.98,523.22,24.79,263865.0,-201425.0,50431.0,-80.87,48.0,X-SC,0.99,86.0,-0.76,0.37,23.28,X40N,NTT,MISC
43,ITC,452.0,-0.09,-1.64,12.28,10.44,24173.0,-3291.0,196847.0,-41.15,46.0,X-LC,1.82,5.0,-0.14,1.46,3.83,X40,NTT,FMCG
36,ICICIPRULI,790.0,0.25,5.68,23.93,30.97,44695.0,10035.0,186773.0,-19.20,60.0,X-MC,2.08,75.0,0.22,1.38,18.95,X40,ATH,INSURANCE
8,AWL,485.0,0.51,-21.31,95.05,53.50,225707.0,-64290.0,237461.0,-61.97,35.0,X-MC,2.09,58.0,-0.28,1.76,5.47,XY24,NTT,FMCG
3,ACC,3906.0,0.91,-24.56,117.60,64.15,211092.0,-58451.0,179500.0,-55.88,40.0,X-SC,2.23,82.0,-0.28,1.33,0.91,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,PGHH,17907.65,0.21,-5.97,42.25,33.76,79783.0,-11985.0,188835.0,-33.72,35.0,X-MC,5.35,57.0,-0.15,1.40,0.21,X40,ATH,FMCG
3,ACC,3906.00,0.91,-24.56,117.60,64.15,211092.0,-58451.0,179500.0,-55.88,40.0,X-SC,2.23,82.0,-0.28,1.33,0.91,XY24,BTT,CEMENT
75,TMPV,600.00,-0.03,-17.88,73.14,42.18,166020.0,-49420.0,226990.0,-26.74,29.0,X-LC,6.38,3.0,-0.30,1.68,0.92,XY24,NTT,AUTO
53,PAGEIND,51605.08,0.01,-9.44,39.21,26.07,58141.0,-15460.0,148280.0,-33.27,33.0,X-MC,12.39,55.0,-0.27,1.10,1.02,X40,ATH,APPARELS
11,BATAINDIA,2096.00,0.01,-38.07,118.33,35.20,94285.0,-48990.0,79680.0,4.40,34.0,X-SC,15.89,92.0,-0.52,0.59,1.21,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,0.38,-29.58,102.85,42.84,47080.0,-19231.0,45775.0,-53.72,42.0,X-SC,36.19,83.0,-0.41,0.34,3.56,XY24,NTT,MISC
15,CAMS,4762.00,0.98,-79.98,523.22,24.79,263865.0,-201425.0,50431.0,-80.87,48.0,X-SC,0.99,86.0,-0.76,0.37,23.28,X40N,NTT,MISC
59,RELAXO,1176.00,0.81,-46.90,186.83,52.31,144009.0,-68080.0,77080.0,-43.47,46.0,X-SC,6.60,91.0,-0.47,0.57,3.18,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.01,-38.07,118.33,35.20,94285.0,-48990.0,79680.0,4.40,34.0,X-SC,15.89,92.0,-0.52,0.59,1.21,X40,NTT,FOOTWEAR
22,DIXON,18931.72,1.93,-14.98,43.00,21.58,51235.0,-20988.0,119151.0,-57.13,34.0,X-MC,7.75,56.0,-0.41,0.88,7.19,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.04,-13.31,37.59,19.28,112738.0,-46040.0,299916.0,-24.27,59.0,X-LC,4.95,1.0,-0.41,2.22,10.87,X40,ATH,IT
41,INFY,2275.00,0.83,8.78,42.37,54.87,144216.0,27486.0,340374.0,-15.29,62.0,X-LC,3.23,2.0,0.19,2.52,17.79,X40,BTT,IT
75,TMPV,600.00,-0.03,-17.88,73.14,42.18,166020.0,-49420.0,226990.0,-26.74,29.0,X-LC,6.38,3.0,-0.30,1.68,0.92,XY24,NTT,AUTO
81,VBL,671.64,-0.04,-3.38,40.42,35.67,123344.0,-10687.0,305155.0,-14.83,57.0,X-LC,4.63,4.0,-0.09,2.26,10.00,X40N,ATH,FMCG
43,ITC,452.00,-0.09,-1.64,12.28,10.44,24173.0,-3291.0,196847.0,-41.15,46.0,X-LC,1.82,5.0,-0.14,1.46,3.83,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,1.50,-37.39,112.25,32.89,54261.0,-28868.0,48339.0,-697.12,55.0,L-MC,5.88,259.0,-0.53,0.36,33.12,XR,NTT,BANKS
6,ASIANTILES,137.00,2.46,-13.83,110.83,81.67,89589.0,-12975.0,80835.0,7266.67,54.0,L-SC,13.69,271.0,-0.14,0.60,56.54,XR,NTT,CERAMICS
50,MASFIN,398.61,1.30,-0.77,22.99,22.05,22353.0,-750.0,97230.0,-14.63,59.0,H-SC,6.99,164.0,-0.03,0.72,41.33,XR,ATH,FINANCE
13,BSOFT,831.70,0.23,-22.23,93.89,50.79,102699.0,-31271.0,109382.0,-0.27,67.0,H-SC,5.96,151.0,-0.30,0.81,27.36,XR,ATH,IT
39,INDIGOPNTS,1408.00,0.33,-12.18,13.90,0.02,21307.0,-21270.0,153289.0,133.82,53.0,M-SC,15.84,234.0,-1.00,1.13,33.33,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,0.23,-22.23,93.89,50.79,102699.0,-31271.0,109382.0,-0.27,67.0,H-SC,5.96,151.0,-0.30,0.81,27.36,XR,ATH,IT
1,ABB,7934.00,0.69,-3.15,50.31,45.57,127469.0,-8251.0,253368.0,-37.78,60.0,H-MC,4.78,120.0,-0.06,1.87,7.81,AR,NTT,ELECTRICAL
0,5PAISA,593.00,0.29,-41.28,92.28,12.90,118390.0,-90202.0,128294.0,88.70,43.0,H-SC,11.84,161.0,-0.76,0.95,6.68,OX40N,NTT,FINANCE
56,RAJESHEXPO,518.00,-1.24,-63.29,172.56,0.07,86906.0,-86814.0,50363.0,1745.96,54.0,L-SC,2.39,268.0,-1.00,0.37,25.26,OX40N,NTT,JEWELLERY
50,MASFIN,398.61,1.30,-0.77,22.99,22.05,22353.0,-750.0,97230.0,-14.63,59.0,H-SC,6.99,164.0,-0.03,0.72,41.33,XR,ATH,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.75
1,25,45.71
2,50,76.67


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.10
MC    30.29
LC    24.62
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.33
X40      22.36
X40N     10.87
XR       10.17
XY25      9.38
AR        9.38
OX40N     7.74
SR        1.03
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.61
X-MC    23.51
X-LC    21.06
M-SC    12.10
X-SC     6.94
H-MC     5.02
L-SC     1.45
M-MC     1.40
M-LC     1.33
H-LC     1.18
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.35,-7.38,42.10
IT,12.76,-16.92,80.12
FINANCE,9.62,-19.27,69.67
ELECTRICAL,6.25,-11.09,51.79
MISC,6.10,-57.37,121.83
PAINTS,6.07,-12.56,29.28
INSURANCE,4.59,-1.83,37.71
PHARMA,4.09,-1.13,33.50
AUTO,3.41,-24.56,77.97


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3208393.0,21
AR,1304156.0,10
XR,1289361.0,13
X40,1018628.0,14
X40N,980371.0,9
OX40N,743346.0,10
XY25,372679.0,6
SR,282212.0,2
MH,82474.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3653204.0,25
M-SC,1421777.0,15
X-MC,1306501.0,16
X-SC,981739.0,8
X-LC,919151.0,11
H-MC,400724.0,3
L-SC,265430.0,3
M-LC,112602.0,1
H-LC,73777.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1274124.0      6
           AR         897567.0      5
M-SC       XY24       812011.0      6
H-SC       XR         771364.0      7
X-SC       X40N       505409.0      3
X-MC       X40        455690.0      7
           XY24       395010.0      3
X-LC       X40        380171.0      5
H-SC       OX40N      345463.0      4
M-SC       OX40N      310977.0      5
X-SC       XY24       293563.0      3
H-SC       SR         282212.0      2
X-MC       X40N       274306.0      4
X-LC       XY24       239594.0      2
H-MC       AR         206633.0      2
X-LC       X40N       200656.0      2
H-MC       XY24       194091.0      1
X-SC       X40        182767.0      2
X-MC       XY25       181495.0      2
L-SC       XR         178524.0      2
M-SC       XR         172610.0      2
           AR         126179.0      2
M-LC       XR         112602.0      1
X-LC       XY25        98730.0      2
L-SC       OX40N       86906.0      1
H-SC       MH          82474.0      1
H-LC       AR          73777.0      1
M-MC       XY25        55630.0      1
L-MC       XR          54261.0      1
L-LC       XY25        36824.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 55.0 seconds
